<h1 align="center">Segmenting and Clustering Neighborhoods in Toronto</h1>

## Part 1 - To scrape the Toronto postal data from Wiki

### 1. Scrape the Wikipedia page

In [123]:
import requests
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plot
from bs4 import BeautifulSoup

# url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969'
html_data = requests.get(url).text
html_tree = BeautifulSoup(html_data, 'html5lib')

### 2. Parse the table data and create a DataFrame with PostalCode, Borough and Neighborhood columns

In [124]:
trs = html_tree.find('tbody').find_all('tr')
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column = ['PostalCode', 'Borough', 'Neighborhood']
toronto_df = pd.DataFrame(columns=column)
# extract raw data from html table to dataframe.
for tr in trs:
    tds = tr.find_all('td')
    if(tds != []):
        toronto_df = toronto_df.append({'PostalCode':tds[0].text.strip(), 'Borough':tds[1].text.strip(), 'Neighborhood':tds[2].text.strip()}, ignore_index=True)
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
# remove Borough value = 'Not assigned'
index_na = toronto_df[(toronto_df['Borough']=='Not assigned')].index
toronto_df.drop(index=index_na, inplace=True)
toronto_df.reset_index(inplace=True)
toronto_df.drop(columns='index', axis=1, inplace=True)
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
if(toronto_df[(toronto_df['Neighborhood']=='Not assigned')].shape[0]!=0):
    print('Found Neighborhood is not assigned; use Borough value')
    toronto_df.loc[toronto_df['Neighborhood']=='Not assigned', 'Neighborhood']=toronto_df.loc[toronto_df['Neighborhood']=='Not assigned', 'Borough'] 
# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
print(toronto_df.shape)
print('The dataframe has {} rows with {} columns'.format(toronto_df.shape[0], toronto_df.shape[1]))
toronto_df

(103, 3)
The dataframe has 103 rows with 3 columns


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


## Part 2 - To retrieve latitude and longitude coordinates of each neighborhood. 
##### Using csv as geospatial input since geocoder is not working at all.

In [125]:
# Using csv as geospatial input since geocoder is not working at all.
import requests
from io import StringIO
path = 'http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
s = requests.get(path).content
geospatial_df = pd.read_csv(StringIO(s.decode("utf-8")))
# geospatial_df.shape
geospatial_df.columns = ['PostalCode', 'Latitude', 'Longitude']
for i in toronto_df['PostalCode'].values:
    latitude = geospatial_df.loc[geospatial_df['PostalCode']==i]['Latitude']
    longitude = geospatial_df.loc[geospatial_df['PostalCode']==i]['Longitude']
    toronto_df.loc[toronto_df['PostalCode']==i, 'Latitude'] = latitude.values[0]
    toronto_df.loc[toronto_df['PostalCode']==i, 'Longitude'] = longitude.values[0]
toronto_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Part 3 - Foursquare API

### Connection information

In [126]:
# @hidden_cell
# API credential
CLIENT_ID = 'MVXHHTRZNOH1RCLYD5VFI2UHLG34E1NN4OURIVOLULWPWWLS' # your Foursquare ID
CLIENT_SECRET = 'REOBTRLX0XUXSFOZMZRO3GJBCXLN1FBAYP22PRAT3FYDCL0X' # your Foursquare Secret
# Foursquare API version
VERSION = '20210314'
RADIUS = 500
LIMIT = 100 # A default Foursquare API limit value

### Explore location through Foursquare API
#### Define a function to get nearby venue

In [127]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Explore each borough that contains the word 'Toronto' and load into dataframe

In [128]:
# Extract borough name contains the word 'Toronto'
target_df = toronto_df[toronto_df['Borough'].str.contains('Toronto')]
venue_df = pd.DataFrame()
venue_df = getNearbyVenues(target_df['Neighborhood'], target_df['Latitude'], target_df['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Runnymede, The Junction, Weston-Pellam Park, Carlton Village
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Har

#### Check dataframe and describe the data

In [129]:
print('There are {} records in venue dataframe'.format(venue_df.shape[0]))
print('and contains {} columns'.format(venue_df.shape[1]))
print(venue_df.columns.values)

print('\nVenue count for each neighborhood:')
print(venue_df[['Neighborhood', 'Venue Category']].groupby('Neighborhood').count())

print('\nCategory counts:')
print(venue_df['Venue Category'].value_counts(ascending=True))

print('\nVenue category - neighborhoods count')
print(venue_df[['Neighborhood', 'Venue Category']].groupby('Venue Category').count())

There are 1619 records in venue dataframe
and contains 7 columns
['Neighborhood' 'Neighborhood Latitude' 'Neighborhood Longitude' 'Venue'
 'Venue Latitude' 'Venue Longitude' 'Venue Category']

Venue count for each neighborhood:
                                                    Venue Category
Neighborhood                                                      
Berczy Park                                                     59
Brockton, Parkdale Village, Exhibition Place                    23
Business reply mail Processing Centre, South Ce...              16
CN Tower, King and Spadina, Railway Lands, Harb...              16
Central Bay Street                                              59
Christie                                                        15
Church and Wellesley                                            78
Commerce Court, Victoria Hotel                                 100
Davisville                                                      34
Davisville North                   

#### Create dataframe with summarized category by each borough-neighborhood

In [130]:
flat_category_df = pd.get_dummies(venue_df[['Venue Category']], prefix='', prefix_sep='')
flat_category_df = flat_category_df.drop(columns=['Neighborhood'])
flat_category_df.insert(0, 'Neighborhood', venue_df['Neighborhood'])
flat_category_df

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1615,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1616,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1617,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Group by Neighborhood and calculate the average appearance of each venue category in a neighborhood

In [131]:
grouped_category_df = flat_category_df.groupby('Neighborhood').mean().reset_index()
grouped_category_df

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.016949,0.00000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.043478
2,"Business reply mail Processing Centre, South C...",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.016949,0.00000,0.000000,0.016949,0.000000,0.016949
5,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.012821,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.012821,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.025641
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.030000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.020000,0.00000,0.000000,0.010000,0.000000,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.029412,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000


#### Get top 5 most common venues for each neighborhood

In [132]:
# Loop for grouped_category_df by Neighborhood
# then transpose the dataframe so that neighborhood is the column
num_top_venues = 5
for hood in grouped_category_df['Neighborhood']:
    print("----"+hood+"----")
#     reverse the dataframe
    temp = grouped_category_df[grouped_category_df['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
#     descending sort and take the first 5 records
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1        Cocktail Bar  0.05
2              Bakery  0.05
3          Restaurant  0.03
4  Seafood Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.13
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3     Yoga Studio  0.04
4    Intersection  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0    Light Rail Station  0.12
1  Gym / Fitness Center  0.06
2         Auto Workshop  0.06
3            Smoke Shop  0.06
4        Farmers Market  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3       Coffee Shop  0.06
4   Harbor / Marina  0.06


----Central Bay Street----
                 venue  freq
0      

#### Function to sort the venues in descending order.

In [133]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create a new dataframe and display the top 10 venues for each neighborhood

In [134]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped_category_df['Neighborhood']

for ind in np.arange(grouped_category_df.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped_category_df.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Pharmacy,Farmers Market,Restaurant,Cheese Shop,Beer Bar,Liquor Store
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Yoga Studio,Grocery Store,Pet Store,Performing Arts Venue,Nightclub,Italian Restaurant,Intersection
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Garden,Brewery,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Skate Park,Auto Workshop,Smoke Shop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Harbor / Marina,Bar
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Thai Restaurant,Middle Eastern Restaurant
5,Christie,Grocery Store,Café,Park,Nightclub,Coffee Shop,Baby Store,Italian Restaurant,Restaurant,Candy Store,Dog Run
6,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Pub,Men's Store,Mediterranean Restaurant,Hotel,Yoga Studio
7,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Gym,Italian Restaurant,American Restaurant,Japanese Restaurant,Deli / Bodega,Seafood Restaurant
8,Davisville,Dessert Shop,Sandwich Place,Pizza Place,Gym,Café,Italian Restaurant,Coffee Shop,Sushi Restaurant,Brewery,Restaurant
9,Davisville North,Gym,Hotel,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,Gym / Fitness Center,General Entertainment,Gluten-free Restaurant


## Part 4 - Cluster the venue of each neighborhood
### Use K-Mean to create venue cluster based on the venue appearance analytics of each neighborhood
#### Neighborhoods are categorized into clusters based on the nature of their appeared venues 

In [135]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = grouped_category_df.drop('Neighborhood', 1)
# manhattan_grouped_clustering
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 4, 0, 0, 4, 0, 0, 0, 0, 4, 0, 2, 0, 0, 4, 4, 0, 4, 0, 3, 0,
       0, 0, 0, 0, 2, 1, 0, 4, 0, 0, 0, 0, 0, 4, 4, 0, 0, 0], dtype=int32)

#### Create a new dataframe that includes the cluster label and the top 10 venues for each neighborhood.

In [136]:
# add clustering labels
# neighborhoods_venues_sorted.drop('Cluster Labels', inplace=True, axis=1)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
# manhattan_merged = manhattan_data
toronto_merged = target_df
# # merge with toronto_df to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged # check the last columns!


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Dessert Shop,Shoe Store,Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,Sushi Restaurant,Yoga Studio,Sandwich Place,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant,Fried Chicken Joint
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Japanese Restaurant,Café,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Hotel,Italian Restaurant,Lingerie Store
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Gastropub,Cosmetics Shop,Cocktail Bar,Gym,Seafood Restaurant,Farmers Market,Hotel,Italian Restaurant
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Health Food Store,Trail,Pub,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Pharmacy,Farmers Market,Restaurant,Cheese Shop,Beer Bar,Liquor Store
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Thai Restaurant,Middle Eastern Restaurant
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564,4,Grocery Store,Café,Park,Nightclub,Coffee Shop,Baby Store,Italian Restaurant,Restaurant,Candy Store,Dog Run
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0,Coffee Shop,Café,Clothing Store,Restaurant,Bakery,Deli / Bodega,Gym,Thai Restaurant,Hotel,Salad Place
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,4,Bakery,Pharmacy,Brewery,Music Venue,Bar,Bank,Middle Eastern Restaurant,Café,Pool,Supermarket


### Create map to show the neighborhoods with cluster label

In [137]:
%pip install folium
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Note: you may need to restart the kernel to use updated packages.


## Part 5 - Verify the neighborhood label
#### Check and see cluster 0 labeled neighborhoods

In [138]:
toronto_merged.loc[toronto_merged['Cluster Labels']==0]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Dessert Shop,Shoe Store,Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,Sushi Restaurant,Yoga Studio,Sandwich Place,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant,Fried Chicken Joint
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Japanese Restaurant,Café,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Hotel,Italian Restaurant,Lingerie Store
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Gastropub,Cosmetics Shop,Cocktail Bar,Gym,Seafood Restaurant,Farmers Market,Hotel,Italian Restaurant
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Pharmacy,Farmers Market,Restaurant,Cheese Shop,Beer Bar,Liquor Store
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Thai Restaurant,Middle Eastern Restaurant
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0,Coffee Shop,Café,Clothing Store,Restaurant,Bakery,Deli / Bodega,Gym,Thai Restaurant,Hotel,Salad Place
36,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,0,Coffee Shop,Aquarium,Café,Hotel,Restaurant,Brewery,Sporting Goods Shop,Italian Restaurant,Fried Chicken Joint,Scenic Lookout
37,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,0,Bar,Vietnamese Restaurant,Restaurant,Café,Asian Restaurant,Men's Store,Cuban Restaurant,Brewery,Record Shop,Pizza Place
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Restaurant,Furniture / Home Store,Spa,Indian Restaurant,Caribbean Restaurant


#### Check and see cluster 1 labeled neighborhoods

In [139]:
toronto_merged.loc[toronto_merged['Cluster Labels']==1]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,M5N,Central Toronto,Roselawn,43.711695,-79.416936,1,Home Service,Garden,Yoga Studio,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


#### Check and see cluster 2 labeled neighborhoods

In [140]:
toronto_merged.loc[toronto_merged['Cluster Labels']==2]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,2,Park,Jewelry Store,Trail,Sushi Restaurant,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
91,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,2,Park,Playground,Trail,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


#### Check and see cluster 3 labeled neighborhoods

In [141]:
toronto_merged.loc[toronto_merged['Cluster Labels']==3]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.38316,3,Gym,Park,Tennis Court,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


#### Check and see cluster 4 labeled neighborhoods

In [142]:
toronto_merged.loc[toronto_merged['Cluster Labels']==4]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Health Food Store,Trail,Pub,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564,4,Grocery Store,Café,Park,Nightclub,Coffee Shop,Baby Store,Italian Restaurant,Restaurant,Candy Store,Dog Run
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,4,Bakery,Pharmacy,Brewery,Music Venue,Bar,Bank,Middle Eastern Restaurant,Café,Pool,Supermarket
47,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,4,Park,Fast Food Restaurant,Sandwich Place,Pet Store,Board Shop,Brewery,Restaurant,Pub,Pizza Place,Movie Theater
61,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Swim School,Bus Line,Business Service,Diner,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
63,M6N,Toronto/York,"Runnymede, The Junction, Weston-Pellam Park, C...",43.673185,-79.487262,4,Grocery Store,Pizza Place,Convenience Store,Brewery,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
69,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,4,Thai Restaurant,Mexican Restaurant,Café,Arts & Crafts Store,Flea Market,Diner,Bar,Italian Restaurant,Bakery,Speakeasy
74,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,4,Café,Sandwich Place,Coffee Shop,Liquor Store,Pizza Place,Indian Restaurant,Metro Station,Pub,BBQ Joint,History Museum
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,4,Light Rail Station,Garden,Brewery,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Skate Park,Auto Workshop,Smoke Shop
